In [2]:
# Trader Behavior Insights — Initial Load & Sanity Check

#Load the Hyperliquid trade data and the Bitcoin Fear & Greed index, parse datetimes, and do an initial data audit.


In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 150)
sns.set(style='whitegrid')


In [4]:
DATA_DIR = os.path.join(os.getcwd(), 'data')   # should point to datascience/data
trades_path = os.path.join(DATA_DIR, 'historical.csv')
fg_path = os.path.join(DATA_DIR, 'fear_greed.csv')

print('Expecting files at:')
print(trades_path)
print(fg_path)

trades = pd.read_csv(trades_path)
fg = pd.read_csv(fg_path)

print('Loaded:')
print('trades:', trades.shape)
print('fg:', fg.shape)

# show first few rows
display(trades.head())
display(fg.head())


Expecting files at:
c:\Users\Aditya\Desktop\datascience\notebooks\data\historical.csv
c:\Users\Aditya\Desktop\datascience\notebooks\data\fear_greed.csv


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\Aditya\\Desktop\\datascience\\notebooks\\data\\historical.csv'

In [5]:
import os
os.listdir(os.path.join(os.path.dirname(os.getcwd()), "data"))


['fear_greed_index.csv', 'historical_data.csv']

In [7]:
import os
import pandas as pd

# Go one level up from /notebooks to reach /datascience
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(ROOT_DIR, 'data')

trades_path = os.path.join(DATA_DIR, 'historical_data.csv')
fg_path = os.path.join(DATA_DIR, 'fear_greed_index.csv')

print("Looking for files at:")
print(trades_path)
print(fg_path)

# Load the data
trades = pd.read_csv(trades_path)
fg = pd.read_csv(fg_path)

print("Loaded successfully:")
print("Trades dataset shape:", trades.shape)
print("Fear & Greed dataset shape:", fg.shape)

display(trades.head())
display(fg.head())


Looking for files at:
c:\Users\Aditya\Desktop\datascience\data\historical_data.csv
c:\Users\Aditya\Desktop\datascience\data\fear_greed_index.csv
Loaded successfully:
Trades dataset shape: (211224, 16)
Fear & Greed dataset shape: (2644, 4)


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [9]:
# Step 5 — Parse datetimes, inspect columns, and basic cleaning
import numpy as np

# 1) Show column names (so you know what to use later)
print("Trades columns:", list(trades.columns))
print("FearGreed columns:", list(fg.columns))
print("-" * 60)

# 2) Parse trade timestamp
# Try common timestamp column names; adapt if your column names differ.
time_col_candidates = ['time', 'timestamp', 'Time', 'date', 'Date']
trade_time_col = None
for c in time_col_candidates:
    if c in trades.columns:
        trade_time_col = c
        break

if trade_time_col is None:
    raise ValueError("No obvious time column found in trades. Check trades.columns above.")

print("Using trade time column:", trade_time_col)
trades[trade_time_col] = pd.to_datetime(trades[trade_time_col], utc=True, errors='coerce')

# 3) Parse Fear&Greed date column
fg_date_col = None
for c in ['Date', 'date', 'day', 'datetime']:
    if c in fg.columns:
        fg_date_col = c
        break

if fg_date_col is None:
    # If no explicit date column, try first column
    fg_date_col = fg.columns[0]
    print("No standard Date column found in FG; using first column:", fg_date_col)
else:
    print("Using FG date column:", fg_date_col)

fg[fg_date_col] = pd.to_datetime(fg[fg_date_col], errors='coerce')

# 4) Create normalized date-only fields for merging
trades['trade_date'] = trades[trade_time_col].dt.date
fg['fg_date'] = fg[fg_date_col].dt.date

# 5) Show missingness and basic stats for key numerical columns if present
print("\nTrades head (preview):")
display(trades.head())
print("\nFear&Greed head (preview):")
display(fg.head())

print("\nMissing values (top 10) in trades:")
print(trades.isnull().sum().sort_values(ascending=False).head(10))

print("\nMissing values (top 10) in fear&greed:")
print(fg.isnull().sum().sort_values(ascending=False).head(10))

# 6) Ensure numeric columns are numeric (common names)
num_cols = ['execution price','execution_price','price','size','closedPnL','closed_pnl','leverage','Leverage']
for col in num_cols:
    if col in trades.columns:
        trades[col] = pd.to_numeric(trades[col], errors='coerce')

# 7) Print basic ranges for numeric columns we found
found_num = [c for c in num_cols if c in trades.columns]
if found_num:
    print("\nNumeric column summaries:")
    display(trades[found_num].describe().T)
else:
    print("\nNo standard numeric columns found among expected names; you'll have to map them manually later.")

# 8) Quick check: how many trades have a matching FG date (non-null after merge preview)
fg_dates_set = set(fg['fg_date'].dropna().unique())
matched_mask = trades['trade_date'].isin(fg_dates_set)
print("\nTrades with sentiment available (approx):", matched_mask.sum(), "/", len(trades))
print("Fraction with sentiment:", matched_mask.mean())


Trades columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
FearGreed columns: ['timestamp', 'value', 'classification', 'date']
------------------------------------------------------------


ValueError: No obvious time column found in trades. Check trades.columns above.

In [10]:
import pandas as pd

trades = pd.read_csv(r"c:\Users\Aditya\Desktop\datascience\data\historical_data.csv")
print(trades.columns)


Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp'],
      dtype='object')


In [11]:
# Step 5 (updated) — parse timestamps and basic cleaning specific to your columns
import pandas as pd
import numpy as np
from datetime import datetime

# reload trades (safe to reload)
trades = pd.read_csv(r"c:\Users\Aditya\Desktop\datascience\data\historical_data.csv")
fg = pd.read_csv(r"c:\Users\Aditya\Desktop\datascience\data\fear_greed_index.csv")

# Select timestamp column: prefer 'Timestamp' then 'Timestamp IST'
if 'Timestamp' in trades.columns and trades['Timestamp'].notna().any():
    time_col = 'Timestamp'
elif 'Timestamp IST' in trades.columns and trades['Timestamp IST'].notna().any():
    time_col = 'Timestamp IST'
else:
    raise ValueError("No usable Timestamp column found in trades.")

print("Using timestamp column:", time_col)

# Parse datetimes (try to infer format)
trades[time_col] = pd.to_datetime(trades[time_col], errors='coerce', infer_datetime_format=True)

# If Timestamp IST exists and Timestamp parsed as NaT for many rows, try parsing IST explicitly:
if time_col == 'Timestamp' and trades[time_col].isna().mean() > 0.2 and 'Timestamp IST' in trades.columns:
    print("Many nulls in 'Timestamp' — trying 'Timestamp IST' fallback parsing...")
    trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], errors='coerce', infer_datetime_format=True)
    # if fallback parsed better, use it
    if trades['Timestamp IST'].notna().sum() > trades[time_col].notna().sum():
        time_col = 'Timestamp IST'
        trades[time_col] = trades['Timestamp IST']
        print("Switched to:", time_col)

# create trade_date for daily merge
trades['trade_date'] = trades[time_col].dt.date

# Convert numeric columns (handle spaces/case)
num_map = {
    'Execution Price': 'execution_price',
    'Size Tokens': 'size_tokens',
    'Size USD': 'size_usd',
    'Closed PnL': 'closed_pnl',
    'Fee': 'fee'
}
for orig, col in num_map.items():
    if orig in trades.columns:
        trades[col] = pd.to_numeric(trades[orig], errors='coerce')
    else:
        trades[col] = np.nan

# Prefer notional from Size USD; fall back to execution_price * size_tokens
if 'size_usd' in trades.columns and trades['size_usd'].notna().sum() > 0:
    trades['notional'] = trades['size_usd']
else:
    trades['notional'] = trades['execution_price'].abs() * trades['size_tokens'].abs()

# closed pnl numeric
trades['closed_pnl'] = trades['closed_pnl']  # already created above

# return pct (safely)
trades['return_pct'] = trades['closed_pnl'] / trades['notional'].replace({0: np.nan})
trades['profitable'] = (trades['closed_pnl'] > 0).astype(int)

# Process FG dates
# detect date column in FG (common names)
fg_date_col = None
for c in ['Date','date','day','datetime']:
    if c in fg.columns:
        fg_date_col = c
        break
if fg_date_col is None:
    fg_date_col = fg.columns[0]

fg[fg_date_col] = pd.to_datetime(fg[fg_date_col], errors='coerce', infer_datetime_format=True)
fg['fg_date'] = fg[fg_date_col].dt.date

# Show outputs and sanity checks
print("\nTrades preview (selected cols):")
display(trades[['Account','Coin','execution_price','size_tokens','size_usd','notional','closed_pnl','return_pct','profitable','trade_date']].head(8))

print("\nNumeric summaries:")
display(trades[['execution_price','size_tokens','size_usd','notional','closed_pnl','return_pct']].describe().T)

print("\nFear&Greed preview:")
display(fg.head())

# How many trades have a sentiment match available?
fg_dates_set = set(fg['fg_date'].dropna().unique())
matched = trades['trade_date'].isin(fg_dates_set)
print("\nTrades with sentiment available:", int(matched.sum()), "/", len(trades))
print("Fraction with sentiment:", matched.mean())

# Print counts of null timestamps parsed
print("\nTimestamp parse stats:")
print("Total rows:", len(trades))
print("Parsed timestamps (non-null):", trades[time_col].notna().sum())
print("Null timestamps:", trades[time_col].isna().sum())


Using timestamp column: Timestamp

Trades preview (selected cols):


C:\Users\Aditya\AppData\Local\Temp\ipykernel_11236\2636550268.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  trades[time_col] = pd.to_datetime(trades[time_col], errors='coerce', infer_datetime_format=True)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11236\2636550268.py:73: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fg[fg_date_col] = pd.to_datetime(fg[fg_date_col], errors='coerce', infer_datetime_format=True)


,Account,Coin,execution_price,size_tokens,size_usd,notional,closed_pnl,return_pct,profitable,trade_date
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,7872.16,0.0,0.0,0,1970-01-01
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,127.68,0.0,0.0,0,1970-01-01
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,1150.63,0.0,0.0,0,1970-01-01
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,1142.04,0.0,0.0,0,1970-01-01
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,69.75,0.0,0.0,0,1970-01-01
5,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9900,1.41,11.27,11.27,0.0,0.0,0,1970-01-01
6,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9934,144.09,1151.77,1151.77,0.0,0.0,0,1970-01-01
7,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,8.0000,34.00,272.00,272.00,0.0,0.0,0,1970-01-01



Numeric summaries:


,count,mean,std,min,25%,50%,75%,max
execution_price,211224.0,11414.723350,29447.654868,4.530000e-06,4.8547,18.280,101.580000,1.090040e+05
size_tokens,211224.0,4623.364979,104272.889530,8.740000e-07,2.9400,32.000,187.902500,1.582244e+07
size_usd,211224.0,5639.451210,36575.138546,0.000000e+00,193.7900,597.045,2058.960000,3.921431e+06
notional,211224.0,5639.451210,36575.138546,0.000000e+00,193.7900,597.045,2058.960000,3.921431e+06
closed_pnl,211224.0,48.749001,919.164828,-1.179901e+05,0.0000,0.000,5.792797,1.353291e+05
return_pct,211181.0,0.018995,0.845813,-3.844064e+02,0.0000,0.000,0.010323,3.403550e+00



Fear&Greed preview:


,timestamp,value,classification,date,fg_date
0,1517463000,30,Fear,2018-02-01,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02,2018-02-02
2,1517635800,40,Fear,2018-02-03,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05,2018-02-05



Trades with sentiment available: 0 / 211224
Fraction with sentiment: 0.0

Timestamp parse stats:
Total rows: 211224
Parsed timestamps (non-null): 211224
Null timestamps: 0


In [12]:
# Step 6 — Merge trades with fear & greed sentiment

# Rename FG columns for clarity if needed
fg = fg.rename(columns={fg_date_col: "fg_datetime"})

# We already created fg['fg_date'] and trades['trade_date'], so merge on date
merged = trades.merge(
    fg[['fg_date', 'fear_and_greed_index'] if 'fear_and_greed_index' in fg.columns else fg.columns],
    left_on='trade_date',
    right_on='fg_date',
    how='left'
)

print("Merged dataset shape:", merged.shape)

print("\nMerged preview:")
display(merged[['trade_date','Account','Coin','return_pct','profitable','fear_and_greed_index' if 'fear_and_greed_index' in merged.columns else merged.columns[-1]]].head())

print("\nNull sentiment rows:", merged['fear_and_greed_index' if 'fear_and_greed_index' in merged.columns else merged.columns[-1]].isna().sum())


Merged dataset shape: (211224, 30)

Merged preview:


,trade_date,Account,Coin,return_pct,profitable,fg_date
0,1970-01-01,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,0.0,0,NaN
1,1970-01-01,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,0.0,0,NaN
2,1970-01-01,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,0.0,0,NaN
3,1970-01-01,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,0.0,0,NaN
4,1970-01-01,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,0.0,0,NaN



Null sentiment rows: 211224


In [15]:
# Detect sentiment column and create features (safe, single cell)
import pandas as pd
import numpy as np

# show merged columns to inspect
print("Merged columns:\n", list(merged.columns))
print("\nLooking for likely sentiment columns...")

# candidates to search for (common names)
candidates = [
    'fear_and_greed_index','fear_greed_index','FearGreedIndex','Score','score',
    'classification','Classification','Value','value','Index','index',
    'fear_greed','Fear_Greed','fear_greed_score','fear_greed_value'
]

sentiment_col = None
for c in candidates:
    if c in merged.columns:
        sentiment_col = c
        break

# if not found, try to find any column that contains 'fear' or 'greed' or 'score' (case-insensitive)
if sentiment_col is None:
    for col in merged.columns:
        low = col.lower()
        if ('fear' in low) or ('greed' in low) or ('score' in low):
            sentiment_col = col
            break

if sentiment_col is None:
    raise KeyError("Couldn't find a sentiment column in merged. Check merged.columns printed above.")

print("Using sentiment column:", sentiment_col)

# Ensure Timestamp column exists (we used it earlier)
time_col = 'Timestamp' if 'Timestamp' in merged.columns else None
if time_col is None:
    # fallback: try 'Timestamp IST' or other
    if 'Timestamp IST' in merged.columns:
        time_col = 'Timestamp IST'
    else:
        raise KeyError("No Timestamp column found in merged. Check merged.columns.")

# Work on a copy
df = merged.copy()

# ensure timestamp parsed as datetime
df[time_col] = pd.to_datetime(df[time_col], errors='coerce')

# sort by Account + time so lag makes sense
df = df.sort_values(['Account', time_col])

# create trade_date if not exists
if 'trade_date' not in df.columns:
    df['trade_date'] = df[time_col].dt.date

# create sentiment numeric column (if classification text exists, keep as-is)
# If the sentiment column is text/classification (e.g., 'Fear'/'Greed'), create a numeric mapping too.
if df[sentiment_col].dtype == object:
    # show unique values (first 20)
    uniq = pd.Series(df[sentiment_col].unique()).astype(str)
    print("First unique values in sentiment column (up to 20):\n", uniq.head(20).tolist())
    # try a simple mapping: Greed=1, Neutral=0, Fear=-1 (only if those values present)
    mapping = {}
    for val in uniq:
        v = str(val).lower()
        if 'greed' in v:
            mapping[val] = 1
        elif 'fear' in v:
            mapping[val] = -1
        elif 'neutral' in v or 'none' in v:
            mapping[val] = 0
    if mapping:
        df['_sentiment_num'] = df[sentiment_col].map(mapping)
        print("Applied text->numeric mapping for sentiment. Example mapping:", mapping)
    else:
        # fallback: leave numeric column as NaN
        df['_sentiment_num'] = np.nan
else:
    # numeric already
    df['_sentiment_num'] = pd.to_numeric(df[sentiment_col], errors='coerce')

# Lagged sentiment per account (use numeric version)
df['sentiment_lag_1'] = df.groupby('Account')['_sentiment_num'].shift(1)
df['sentiment_lag_2'] = df.groupby('Account')['_sentiment_num'].shift(2)

# Rolling 7 (per account) on numeric sentiment
df['sentiment_roll_7'] = df.groupby('Account')['_sentiment_num'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)

# side encoding (safe mapping)
if 'Side' in df.columns:
    df['side_num'] = df['Side'].map({'Buy': 1, 'Sell': -1}).fillna(0)
else:
    df['side_num'] = 0

# ensure notional exists (we computed earlier as 'notional' possibly). If different name, try common ones
if 'notional' not in df.columns:
    if 'Size USD' in df.columns:
        df['notional'] = pd.to_numeric(df['Size USD'], errors='coerce')
    elif 'size_usd' in df.columns:
        df['notional'] = pd.to_numeric(df['size_usd'], errors='coerce')
    else:
        # try compute from Execution Price * Size Tokens
        if 'Execution Price' in df.columns and 'Size Tokens' in df.columns:
            df['notional'] = pd.to_numeric(df['Execution Price'], errors='coerce').abs() * pd.to_numeric(df['Size Tokens'], errors='coerce').abs()
        else:
            df['notional'] = np.nan

# closed pnl numeric (try common names)
if 'Closed PnL' in df.columns:
    df['closed_pnl'] = pd.to_numeric(df['Closed PnL'], errors='coerce')
elif 'closed_pnl' not in df.columns:
    df['closed_pnl'] = pd.to_numeric(df.get('Closed PnL', df.get('closed_pnl', np.nan)), errors='coerce')

# return pct
df['return_pct'] = df['closed_pnl'] / df['notional'].replace({0: np.nan})

# profitable target
df['profitable'] = (df['closed_pnl'] > 0).astype(int)

# normalized size_usd
if 'notional' in df.columns:
    df['size_usd_norm'] = (df['notional'] - df['notional'].mean()) / df['notional'].std()

# Prepare final ML features list (adjustable)
features = ['_sentiment_num','sentiment_lag_1','sentiment_lag_2','sentiment_roll_7','side_num','size_usd_norm']
ml_df = df.dropna(subset=['_sentiment_num','profitable'] + [c for c in ['size_usd_norm'] if c in df.columns], how='any').copy()

print("\nML dataframe shape (rows with required features):", ml_df.shape)
print("\nFeature sample:")
display(ml_df[features + ['profitable','Account','trade_date']].head(10))


Merged columns:
 ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp', 'trade_date', 'execution_price', 'size_tokens', 'size_usd', 'closed_pnl', 'fee', 'notional', 'return_pct', 'profitable', 'timestamp', 'value', 'classification', 'fg_datetime', 'fg_date']

Looking for likely sentiment columns...
Using sentiment column: classification
First unique values in sentiment column (up to 20):
 ['nan']

ML dataframe shape (rows with required features): (0, 36)

Feature sample:


,_sentiment_num,sentiment_lag_1,sentiment_lag_2,sentiment_roll_7,side_num,size_usd_norm,profitable,Account,trade_date


In [16]:
# Fix sentiment source and recreate features (copy & run)
import pandas as pd
import numpy as np

df = merged.copy()

# Prefer numeric 'value' column for sentiment (common in Fear&Greed data)
if 'value' in df.columns:
    df['_sentiment_num'] = pd.to_numeric(df['value'], errors='coerce')
    print("Using 'value' column as numeric sentiment where available.")
else:
    df['_sentiment_num'] = np.nan

# If numeric not available for some rows, try mapping 'classification' text
if df['_sentiment_num'].isna().any() and 'classification' in df.columns:
    # show unique classification values
    uniq = pd.Series(df['classification'].unique()).astype(str)
    print("Classification uniques (first 20):", uniq.head(20).tolist())
    # mapping
    mapping = {}
    for val in uniq:
        v = str(val).lower()
        if 'greed' in v:
            mapping[val] = 1
        elif 'fear' in v:
            mapping[val] = -1
        elif 'neutral' in v or 'none' in v:
            mapping[val] = 0
    if mapping:
        df['_sentiment_num'] = df['_sentiment_num'].fillna(df['classification'].map(mapping))
        print("Applied classification->numeric mapping (example):", mapping)
    else:
        print("No useful text mapping for classification found.")

# Count how many rows have sentiment now
n_with_sent = df['_sentiment_num'].notna().sum()
print(f"Rows with numeric sentiment after fix: {n_with_sent} / {len(df)} ({n_with_sent/len(df):.2%})")

# Ensure timestamp column exists and is datetime
ts_col = 'Timestamp' if 'Timestamp' in df.columns else ('Timestamp IST' if 'Timestamp IST' in df.columns else None)
if ts_col is None:
    raise KeyError("No timestamp column found.")
df[ts_col] = pd.to_datetime(df[ts_col], errors='coerce')

# sort for lagging
df = df.sort_values(['Account', ts_col])

# create lag and rolling features on _sentiment_num (per account)
df['sentiment_lag_1'] = df.groupby('Account')['_sentiment_num'].shift(1)
df['sentiment_lag_2'] = df.groupby('Account')['_sentiment_num'].shift(2)
df['sentiment_roll_7'] = df.groupby('Account')['_sentiment_num'].rolling(7, min_periods=1).mean().reset_index(0, drop=True)

# side encoding
df['side_num'] = df['Side'].map({'Buy':1,'Sell':-1}).fillna(0)

# ensure notional exists (you had it earlier)
if 'notional' not in df.columns:
    if 'Size USD' in df.columns:
        df['notional'] = pd.to_numeric(df['Size USD'], errors='coerce')
    else:
        df['notional'] = (pd.to_numeric(df.get('Execution Price', np.nan), errors='coerce').abs()
                         * pd.to_numeric(df.get('Size Tokens', np.nan), errors='coerce').abs())

# closed_pnl numeric
df['closed_pnl'] = pd.to_numeric(df.get('Closed PnL', df.get('closed_pnl', np.nan)), errors='coerce')

# target and return
df['return_pct'] = df['closed_pnl'] / df['notional'].replace({0: np.nan})
df['profitable'] = (df['closed_pnl'] > 0).astype(int)

# normalized size
df['size_usd_norm'] = (df['notional'] - df['notional'].mean()) / df['notional'].std()

# Build ML dataframe: require numeric sentiment + notional + profitable
ml_df = df.dropna(subset=['_sentiment_num','notional','profitable']).copy()

print("\nML dataframe shape (rows kept):", ml_df.shape)
print("Example feature summary:")
display(ml_df[['_sentiment_num','sentiment_lag_1','sentiment_lag_2','sentiment_roll_7','side_num','size_usd_norm','profitable']].head(10))

# Basic distributions to inspect
print("\nSentiment numeric summary:")
display(ml_df['_sentiment_num'].describe())

print("\nReturn pct summary (trimmed):")
display(ml_df['return_pct'].describe())

# Also show how many unique accounts are available for ML
print("\nUnique accounts in ML dataset:", ml_df['Account'].nunique())


Using 'value' column as numeric sentiment where available.
Classification uniques (first 20): ['nan']
No useful text mapping for classification found.
Rows with numeric sentiment after fix: 0 / 211224 (0.00%)

ML dataframe shape (rows kept): (0, 36)
Example feature summary:


,_sentiment_num,sentiment_lag_1,sentiment_lag_2,sentiment_roll_7,side_num,size_usd_norm,profitable



Sentiment numeric summary:


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: _sentiment_num, dtype: float64


Return pct summary (trimmed):


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: return_pct, dtype: float64


Unique accounts in ML dataset: 0


In [17]:
print(df['value'].head(20))
print(df['value'].dtype)
print(df['classification'].unique()[:20])


14229   NaN
14230   NaN
14231   NaN
14232   NaN
14233   NaN
14234   NaN
14235   NaN
14236   NaN
14237   NaN
14238   NaN
14239   NaN
14240   NaN
14241   NaN
14242   NaN
14243   NaN
14244   NaN
14245   NaN
14246   NaN
14247   NaN
14248   NaN
Name: value, dtype: float64
float64
[nan]


In [18]:
# Inspect the Fear & Greed table to see if any numeric sentiment exists
print("FG columns:", list(fg.columns))
print("\nFG head (first 10 rows):")
display(fg.head(10))

print("\nFG info():")
display(fg.info())

print("\nValue column stats (non-null counts and sample non-null rows):")
print("dtype:", fg.get('value').dtype if 'value' in fg.columns else 'no value col')
print("non-null count:", fg['value'].notna().sum() if 'value' in fg.columns else 0)
if 'value' in fg.columns and fg['value'].notna().sum() > 0:
    display(fg.loc[fg['value'].notna(), ['Date','value','classification']].head(20))

print("\nClassification unique values (up to 50):")
if 'classification' in fg.columns:
    print(pd.Series(fg['classification'].unique()).astype(str)[:50].tolist())
else:
    print("No 'classification' column.")
    
print("\nDate range in FG (fg_date):")
if 'fg_date' in fg.columns:
    print("min:", fg['fg_date'].min(), "max:", fg['fg_date'].max())
else:
    print("fg_date not present")


FG columns: ['timestamp', 'value', 'classification', 'fg_datetime', 'fg_date']

FG head (first 10 rows):


,timestamp,value,classification,fg_datetime,fg_date
0,1517463000,30,Fear,2018-02-01,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02,2018-02-02
2,1517635800,40,Fear,2018-02-03,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05,2018-02-05
5,1517895000,8,Extreme Fear,2018-02-06,2018-02-06
6,1517981400,36,Fear,2018-02-07,2018-02-07
7,1518067800,30,Fear,2018-02-08,2018-02-08
8,1518154200,44,Fear,2018-02-09,2018-02-09
9,1518240600,54,Neutral,2018-02-10,2018-02-10



FG info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   timestamp       2644 non-null   int64         
 1   value           2644 non-null   int64         
 2   classification  2644 non-null   object        
 3   fg_datetime     2644 non-null   datetime64[ns]
 4   fg_date         2644 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 103.4+ KB


None


Value column stats (non-null counts and sample non-null rows):
dtype: int64
non-null count: 2644


KeyError: "['Date'] not in index"

In [19]:
# STEP 1 — Clean and align dates
df['trade_date'] = pd.to_datetime(df['Timestamp']).dt.date
fg['fg_date_clean'] = pd.to_datetime(fg['fg_date']).dt.date

# STEP 2 — Merge on trade_date (left join)
df_merged = df.merge(
    fg[['fg_date_clean', 'value', 'classification']],
    left_on='trade_date',
    right_on='fg_date_clean',
    how='left'
)

print("Rows in merged DF:", df_merged.shape)
print("Rows with sentiment:", df_merged['value'].notna().sum())
print("Sample sentiment rows:")
display(df_merged[['Timestamp', 'trade_date', 'value', 'classification']].head(10))


Rows in merged DF: (211224, 39)


KeyError: 'value'

In [20]:
# Safe reload + robust merge (run this cell exactly)
import os
import pandas as pd
from datetime import datetime

ROOT = os.path.dirname(os.getcwd())   # notebooks is current dir, parent is project root
DATA_DIR = os.path.join(ROOT, 'data')
trades_path = os.path.join(DATA_DIR, 'historical_data.csv')
fg_path = os.path.join(DATA_DIR, 'fear_greed_index.csv')

# 1) reload fresh
trades = pd.read_csv(trades_path)
fg = pd.read_csv(fg_path)

# 2) show columns (so you can confirm)
print("Trades columns:\n", list(trades.columns))
print("\nFG columns:\n", list(fg.columns))

# 3) detect best FG sentiment column (numeric preferred)
possible_numeric = ['value','Value','score','Score','index','Index','fg_value','fg_score']
sent_col = None
for c in possible_numeric:
    if c in fg.columns:
        sent_col = c
        break

# if not found, try to find any column name containing 'val' 'score' 'index' case-insensitive
if sent_col is None:
    for c in fg.columns:
        if any(k in c.lower() for k in ['val','score','index']):
            sent_col = c
            break

# fallback to classification if numeric absent
if sent_col is None and 'classification' in fg.columns:
    sent_col = 'classification'

print("\nDetected sentiment column (will use):", sent_col)

# 4) create fg_date_clean from any plausible date/datetime column
date_col = None
for c in ['fg_datetime','fg_date','Date','date','timestamp','datetime']:
    if c in fg.columns:
        date_col = c
        break
if date_col is None:
    # try first column
    date_col = fg.columns[0]
print("Detected FG date column:", date_col)

# convert to date
fg['fg_date_clean'] = pd.to_datetime(fg[date_col], errors='coerce').dt.date

# create trade_date in trades
# prefer 'Timestamp' then 'Timestamp IST' else try first datetime-like
if 'Timestamp' in trades.columns:
    trades['trade_date'] = pd.to_datetime(trades['Timestamp'], errors='coerce').dt.date
elif 'Timestamp IST' in trades.columns:
    trades['trade_date'] = pd.to_datetime(trades['Timestamp IST'], errors='coerce').dt.date
else:
    # try to parse first column that looks like datetime
    parsed = False
    for c in trades.columns:
        if 'time' in c.lower() or 'date' in c.lower() or 'timestamp' in c.lower():
            try:
                trades['trade_date'] = pd.to_datetime(trades[c], errors='coerce').dt.date
                parsed = True
                print("Parsed trade_date from column:", c)
                break
            except Exception:
                pass
    if not parsed:
        raise RuntimeError("Could not find or parse a timestamp column in trades.")

# 5) merge on date
if sent_col is None:
    raise RuntimeError("No sentiment column detected in FG. Check fg.columns printed above.")

# choose which FG columns to bring in
fg_keep = ['fg_date_clean', sent_col]
fg_small = fg[fg_keep].drop_duplicates(subset=['fg_date_clean'])

merged = trades.merge(fg_small, left_on='trade_date', right_on='fg_date_clean', how='left')

# 6) reporting
print("\nMerged shape:", merged.shape)
num_with_sent = merged[sent_col].notna().sum()
print("Rows with sentiment (non-null '{}'): {} / {}  ({:.2%})".format(sent_col, num_with_sent, len(merged), num_with_sent/len(merged)))

# show sample of merged columns to verify
sample_cols = ['trade_date','Timestamp'] if 'Timestamp' in merged.columns else ['trade_date']
sample_cols += [sent_col]
print("\nSample merged rows (first 10):")
display(merged[sample_cols].head(10))

# expose merged to notebook session
merged.to_csv(os.path.join(DATA_DIR, 'merged_debug_sample.csv'), index=False)
print("\nA saved sample 'merged_debug_sample.csv' was written to the data folder for inspection.")


Trades columns:
 ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']

FG columns:
 ['timestamp', 'value', 'classification', 'date']

Detected sentiment column (will use): value
Detected FG date column: date

Merged shape: (211224, 19)
Rows with sentiment (non-null 'value'): 0 / 211224  (0.00%)

Sample merged rows (first 10):


,trade_date,Timestamp,value
0,1970-01-01,1.730000e+12,NaN
1,1970-01-01,1.730000e+12,NaN
2,1970-01-01,1.730000e+12,NaN
3,1970-01-01,1.730000e+12,NaN
4,1970-01-01,1.730000e+12,NaN
5,1970-01-01,1.730000e+12,NaN
6,1970-01-01,1.730000e+12,NaN
7,1970-01-01,1.730000e+12,NaN
8,1970-01-01,1.730000e+12,NaN
9,1970-01-01,1.730000e+12,NaN



A saved sample 'merged_debug_sample.csv' was written to the data folder for inspection.


In [21]:
import pandas as pd
import os

ROOT = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(ROOT, "data")

trades = pd.read_csv(os.path.join(DATA_DIR, "historical_data.csv"))
fg = pd.read_csv(os.path.join(DATA_DIR, "fear_greed_index.csv"))

# ---- FIX TRADE TIMESTAMP ----
# Convert UNIX ms → datetime
trades['Timestamp_dt'] = pd.to_datetime(trades['Timestamp'], unit='ms', errors='coerce')
trades['trade_date'] = trades['Timestamp_dt'].dt.date

print("Trade date sample:", trades['trade_date'].head(10))

# ---- Fix FG Date ----
fg['fg_date_clean'] = pd.to_datetime(fg['date'], errors='coerce').dt.date

# ---- Merge ----
merged = trades.merge(
    fg[['fg_date_clean', 'value']],
    left_on='trade_date',
    right_on='fg_date_clean',
    how='left'
)

print("\nMerged shape:", merged.shape)
non_null = merged['value'].notna().sum()
print("Rows with sentiment:", non_null, "/", len(merged), f"({non_null/len(merged):.2%})")

print("\nSample merged rows:")
display(merged[['Timestamp_dt','trade_date','value']].head(10))


Trade date sample: 0    2024-10-27
1    2024-10-27
2    2024-10-27
3    2024-10-27
4    2024-10-27
5    2024-10-27
6    2024-10-27
7    2024-10-27
8    2024-10-27
9    2024-10-27
Name: trade_date, dtype: object

Merged shape: (211224, 20)
Rows with sentiment: 184263 / 211224 (87.24%)

Sample merged rows:


,Timestamp_dt,trade_date,value
0,2024-10-27 03:33:20,2024-10-27,74.0
1,2024-10-27 03:33:20,2024-10-27,74.0
2,2024-10-27 03:33:20,2024-10-27,74.0
3,2024-10-27 03:33:20,2024-10-27,74.0
4,2024-10-27 03:33:20,2024-10-27,74.0
5,2024-10-27 03:33:20,2024-10-27,74.0
6,2024-10-27 03:33:20,2024-10-27,74.0
7,2024-10-27 03:33:20,2024-10-27,74.0
8,2024-10-27 03:33:20,2024-10-27,74.0
9,2024-10-27 03:33:20,2024-10-27,74.0


In [22]:
import pandas as pd
import numpy as np

df = merged.copy()

# Ensure sorted by account and time
df = df.sort_values(["Account", "Timestamp_dt"])

# ----------------------
# 1. Lagged sentiment
# ----------------------
df['sentiment_lag_1'] = df.groupby('Account')['value'].shift(1)
df['sentiment_lag_2'] = df.groupby('Account')['value'].shift(2)
df['sentiment_lag_3'] = df.groupby('Account')['value'].shift(3)

# ----------------------
# 2. Rolling averages
# ----------------------
df['sentiment_ma_7'] = df.groupby('Account')['value'].transform(lambda x: x.rolling(7, min_periods=1).mean())
df['sentiment_ma_14'] = df.groupby('Account')['value'].transform(lambda x: x.rolling(14, min_periods=1).mean())

# ----------------------
# 3. Trader performance features
# ----------------------
df['profit'] = df['Closed PnL']
df['profitable'] = (df['profit'] > 0).astype(int)

# notional = price × size
df['notional_usd'] = df['Execution Price'] * df['Size Tokens']

# Buy = 1, Sell = 0
df['is_buy'] = (df['Side'].str.lower() == 'buy').astype(int)

# ----------------------
# 4. Drop rows missing sentiment (only ~12%)
# ----------------------
df_ml = df.dropna(subset=['value'])

print("Final ML dataset shape:", df_ml.shape)
print(df_ml[['Timestamp_dt','value','sentiment_lag_1','sentiment_ma_7','profit','profitable']].head())


Final ML dataset shape: (184263, 29)
             Timestamp_dt  value  sentiment_lag_1  sentiment_ma_7  profit  profitable
14229 2024-10-27 03:33:20   74.0              NaN            74.0     0.0           0
14230 2024-10-27 03:33:20   74.0             74.0            74.0     0.0           0
14231 2024-10-27 03:33:20   74.0             74.0            74.0     0.0           0
14232 2024-10-27 03:33:20   74.0             74.0            74.0     0.0           0
14233 2024-10-27 03:33:20   74.0             74.0            74.0     0.0           0


In [23]:
import pandas as pd
import numpy as np

df = df_ml.copy()

# -------------------------
# 1. Profitability vs Sentiment
# -------------------------
sentiment_profit = df.groupby('value')['profitable'].mean()

print("\n==== Profitability vs Sentiment ====\n")
print(sentiment_profit.head(20))

# -------------------------
# 2. Avg profit amount vs sentiment
# -------------------------
sentiment_pnl = df.groupby('value')['profit'].mean()

print("\n==== Avg Profit Amount vs Sentiment ====\n")
print(sentiment_pnl.head(20))

# -------------------------
# 3. Buy Ratio at each sentiment level
# -------------------------
buy_ratio = df.groupby('value')['is_buy'].mean()

print("\n==== Buy Ratio vs Sentiment ====\n")
print(buy_ratio.head(20))

# -------------------------
# 4. Summary stats by sentiment bucket
# Fear = 0–49, Neutral = 50–59, Greed = 60+
# -------------------------
df['sentiment_group'] = pd.cut(
    df['value'],
    bins=[0, 49, 59, 100],
    labels=['Fear', 'Neutral', 'Greed']
)

summary = df.groupby('sentiment_group')[['profit', 'profitable', 'is_buy']].mean()

print("\n==== Sentiment Group Summary ====\n")
print(summary)



==== Profitability vs Sentiment ====

value
44.0    0.415146
50.0    0.317182
59.0    0.000000
69.0    0.274641
74.0    0.451605
84.0    0.490089
Name: profitable, dtype: float64

==== Avg Profit Amount vs Sentiment ====

value
44.0    50.047622
50.0    22.229713
59.0     0.000000
69.0     0.148807
74.0    90.504272
84.0    25.418772
Name: profit, dtype: float64

==== Buy Ratio vs Sentiment ====

value
44.0    0.493617
50.0    0.490828
59.0    1.000000
69.0    0.469856
74.0    0.423569
84.0    0.484200
Name: is_buy, dtype: float64

==== Sentiment Group Summary ====

                    profit  profitable    is_buy
sentiment_group                                 
Fear             50.047622    0.415146  0.493617
Neutral          22.220378    0.317049  0.491041
Greed            77.843647    0.453524  0.434448


C:\Users\Aditya\AppData\Local\Temp\ipykernel_11236\722719111.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby('sentiment_group')[['profit', 'profitable', 'is_buy']].mean()


In [24]:
import os
from pathlib import Path

# Adjust these names if you used different variable names in your notebook
# merged  -> full merged trades + sentiment
# df_ml   -> final ML-ready DataFrame (rows with sentiment)

# ------------- create folder structure -----------------
ROOT = os.path.dirname(os.getcwd())   # parent of notebooks
OUT_ROOT = os.path.join(ROOT, "ds_aditya_sawant")

csv_dir = os.path.join(OUT_ROOT, "csv_files")
out_dir = os.path.join(OUT_ROOT, "outputs")

os.makedirs(csv_dir, exist_ok=True)
os.makedirs(out_dir, exist_ok=True)

print("Created folders at:", OUT_ROOT)
print("csv_files:", csv_dir)
print("outputs:", out_dir)

# ------------- save CSVs -----------------
# Save merged dataset (full)
merged_path = os.path.join(csv_dir, "merged_trades.csv")
merged.to_csv(merged_path, index=False)
print("Saved merged dataset ->", merged_path)

# Save final ML dataset (only rows with sentiment)
final_ml_path = os.path.join(csv_dir, "final_ml_dataset.csv")
df_ml.to_csv(final_ml_path, index=False)
print("Saved final ML dataset ->", final_ml_path)

# Save a small sample: top accounts by avg return (helpful for report)
try:
    sample = df_ml.groupby('Account').agg(
        trades_count=('profitable','count'),
        win_rate=('profitable','mean'),
        avg_profit=('profit','mean'),
        avg_notional=('notional_usd','mean')
    ).reset_index().sort_values('avg_profit', ascending=False).head(50)
    sample_path = os.path.join(csv_dir, "top_accounts_sample.csv")
    sample.to_csv(sample_path, index=False)
    print("Saved sample top_accounts ->", sample_path)
except Exception as e:
    print("Could not create top_accounts sample (no group columns?), error:", e)

# ------------- write README.md -----------------
readme_text = f"""# ds_aditya_sawant

This repository contains the assignment for the Junior Data Scientist — Trader Behavior Insights.

Structure:
- csv_files/: merged and processed CSV files
- outputs/: charts and images (to be generated)
- notebook_1.ipynb: the main Google Colab notebook (you should upload the .ipynb here)
- ds_report.pdf: final summarized insights (to be generated)
"""

with open(os.path.join(OUT_ROOT, "README.md"), "w", encoding="utf-8") as f:
    f.write(readme_text)
print("Wrote README.md")

# ------------- write starter ds_report.md -----------------
report_md = f"""# Trader Behavior Insights — Draft Report

**Candidate:** Aditya (Gaami) Sawant  
**Dataset:** Hyperliquid historical trades + Bitcoin Fear & Greed Index

## Objective
Analyze how trader behavior (profitability, risk, volume, leverage) aligns with market sentiment (Fear vs Greed).

## Key preliminary findings (from EDA)
- Greed days (value ≥60) show **higher average profit** and slightly **higher probability of profitable trades** compared to Fear days in this dataset.
- Example aggregated numbers (approx):
  - Fear: avg profit ≈ 50 USD, win rate ≈ 41.5%
  - Neutral: avg profit ≈ 22 USD, win rate ≈ 31.7%
  - Greed: avg profit ≈ 77.8 USD, win rate ≈ 45.4%
- Buy ratio is slightly **higher in Fear** and lower in Greed (shorting/less buys in Greed).

## Next steps performed in notebook_1.ipynb
1. Data loading, parsing timestamps (Unix ms), merging daily sentiment.
2. Feature engineering: return_pct, notional_usd, lagged sentiment (1–3 days), rolling sentiment (7/14 days).
3. EDA and statistical tests (ANOVA / Kruskal-Wallis recommended).
4. Predictive modeling (Logistic Regression / Random Forest) to predict profitable trades.
5. Actionable recommendations based on results.

## Recommendations (short)
- Use sentiment as a risk overlay: reduce leverage on extreme Greed days.
- Monitor 3-day sentiment drops as short-term reversal signals.
- Further work: incorporate BTC price volatility, order-level holding times, and trader clustering.

(Full details, code, and charts available in notebook_1.ipynb)
"""

with open(os.path.join(OUT_ROOT, "ds_report.md"), "w", encoding="utf-8") as f:
    f.write(report_md)
print("Wrote ds_report.md (draft)")

# ------------- done -------------
print("\nAll files created. Next: we will generate the required output charts (png) and then convert ds_report.md to PDF.")


Created folders at: c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant
csv_files: c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\csv_files
outputs: c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs
Saved merged dataset -> c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\csv_files\merged_trades.csv
Saved final ML dataset -> c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\csv_files\final_ml_dataset.csv
Saved sample top_accounts -> c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\csv_files\top_accounts_sample.csv
Wrote README.md
Wrote ds_report.md (draft)

All files created. Next: we will generate the required output charts (png) and then convert ds_report.md to PDF.


In [25]:
# Step B — Generate and save output charts (run this cell)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

ROOT = os.path.dirname(os.getcwd())
OUT_ROOT = os.path.join(ROOT, "ds_aditya_sawant")
OUT_DIR = os.path.join(OUT_ROOT, "outputs")
os.makedirs(OUT_DIR, exist_ok=True)

# Use df_ml if available, else try merged with value present
if 'df_ml' in globals():
    df = df_ml.copy()
elif 'df' in globals():
    df = df.copy()
elif 'merged' in globals():
    df = merged.copy()
else:
    raise RuntimeError("Could not find df_ml, df, or merged in the notebook namespace. Load merged data first.")

# Ensure necessary cols
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['profit'] = pd.to_numeric(df.get('profit', df.get('Closed PnL', df.get('closed_pnl'))), errors='coerce')
df['profitable'] = (df['profit'] > 0).astype(int)
df['is_buy'] = df.get('is_buy', (df.get('Side', '').str.lower() == 'buy').astype(int))

# Create sentiment groups
df['sentiment_group'] = pd.cut(df['value'], bins=[-1,49,59,100], labels=['Fear','Neutral','Greed'])

sns.set(style='whitegrid')

# Chart 1: Profitability rate by sentiment group (bar)
plt.figure(figsize=(6,4))
grp = df.groupby('sentiment_group')['profitable'].mean().reindex(['Fear','Neutral','Greed'])
sns.barplot(x=grp.index, y=grp.values)
plt.ylabel('Probability of Profitable Trade')
plt.title('Profitability Rate by Sentiment Group')
plt.ylim(0,1)
plt.tight_layout()
p1 = os.path.join(OUT_DIR, 'profitability_by_sentiment.png')
plt.savefig(p1, dpi=150)
plt.close()

# Chart 2: Average profit (USD) by sentiment group (bar)
plt.figure(figsize=(6,4))
grp2 = df.groupby('sentiment_group')['profit'].mean().reindex(['Fear','Neutral','Greed'])
sns.barplot(x=grp2.index, y=grp2.values)
plt.ylabel('Average Profit (USD)')
plt.title('Average Profit by Sentiment Group')
plt.tight_layout()
p2 = os.path.join(OUT_DIR, 'avg_profit_by_sentiment.png')
plt.savefig(p2, dpi=150)
plt.close()

# Chart 3: Boxplot of return_pct by sentiment group
if 'return_pct' in df.columns:
    plt.figure(figsize=(8,5))
    sns.boxplot(data=df, x='sentiment_group', y='return_pct', order=['Fear','Neutral','Greed'])
    plt.ylim(np.percentile(df['return_pct'].dropna(),1), np.percentile(df['return_pct'].dropna(),99))
    plt.title('Return % by Sentiment Group (1-99 pct shown)')
    plt.tight_layout()
    p3 = os.path.join(OUT_DIR, 'returnpct_box_by_sentiment.png')
    plt.savefig(p3, dpi=150)
    plt.close()
else:
    p3 = None

# Chart 4: Buy ratio by sentiment group
plt.figure(figsize=(6,4))
buy_ratio = df.groupby('sentiment_group')['is_buy'].mean().reindex(['Fear','Neutral','Greed'])
sns.barplot(x=buy_ratio.index, y=buy_ratio.values)
plt.ylabel('Buy Ratio (fraction)')
plt.title('Buy Ratio by Sentiment Group')
plt.ylim(0,1)
plt.tight_layout()
p4 = os.path.join(OUT_DIR, 'buy_ratio_by_sentiment.png')
plt.savefig(p4, dpi=150)
plt.close()

# Chart 5: Sentiment value distribution (histogram)
plt.figure(figsize=(8,4))
sns.histplot(df['value'].dropna(), bins=20, kde=False)
plt.xlabel('Fear & Greed Score')
plt.title('Distribution of Fear & Greed Scores in Dataset')
plt.tight_layout()
p5 = os.path.join(OUT_DIR, 'sentiment_distribution.png')
plt.savefig(p5, dpi=150)
plt.close()

# Chart 6: Top 15 accounts by avg profit (bar)
try:
    acct = df.groupby('Account').agg(avg_profit=('profit','mean'), trades=('profit','count')).reset_index()
    top = acct[acct['trades']>=5].sort_values('avg_profit', ascending=False).head(15)
    plt.figure(figsize=(10,5))
    sns.barplot(data=top, x='avg_profit', y='Account')
    plt.xlabel('Average Profit (USD)')
    plt.title('Top 15 Accounts by Avg Profit (min 5 trades)')
    plt.tight_layout()
    p6 = os.path.join(OUT_DIR, 'top15_accounts_avg_profit.png')
    plt.savefig(p6, dpi=150)
    plt.close()
except Exception as e:
    p6 = None
    print("Could not create top accounts chart:", e)

saved = [p for p in [p1,p2,p3,p4,p5,p6] if p]
print("Saved plots:")
for s in saved:
    print("-", s)

# expose a list for later steps
plots_saved = saved


C:\Users\Aditya\AppData\Local\Temp\ipykernel_11236\4097272859.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df.groupby('sentiment_group')['profitable'].mean().reindex(['Fear','Neutral','Greed'])
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11236\4097272859.py:48: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp2 = df.groupby('sentiment_group')['profit'].mean().reindex(['Fear','Neutral','Greed'])
C:\Users\Aditya\AppData\Local\Temp\ipykernel_11236\4097272859.py:72: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=Fa

Saved plots:
- c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs\profitability_by_sentiment.png
- c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs\avg_profit_by_sentiment.png
- c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs\buy_ratio_by_sentiment.png
- c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs\sentiment_distribution.png
- c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs\top15_accounts_avg_profit.png


In [26]:
# Create ds_report.pdf by embedding images and text from ds_report.md
# Installs reportlab if missing, then writes a simple formatted PDF.

import os
import textwrap
from pathlib import Path
ROOT = os.path.dirname(os.getcwd())
OUT_ROOT = os.path.join(ROOT, "ds_aditya_sawant")
MD_PATH = os.path.join(OUT_ROOT, "ds_report.md")
OUT_PDF = os.path.join(OUT_ROOT, "ds_report.pdf")
OUT_DIR = os.path.join(OUT_ROOT, "outputs")

# install reportlab if not present
import importlib, sys
pkg = importlib.util.find_spec("reportlab")
if pkg is None:
    print("Installing reportlab...")
    !pip install reportlab --quiet

from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

def write_pdf_from_md(md_path, out_pdf, images_dir):
    # read markdown
    with open(md_path, 'r', encoding='utf-8') as f:
        md = f.read()
    # naive markdown -> paragraphs (keeps headings and lists as text)
    lines = md.splitlines()
    # prepare canvas
    c = canvas.Canvas(out_pdf, pagesize=A4)
    width, height = A4
    margin = 20*mm
    x = margin
    y = height - margin
    max_width = width - 2*margin
    line_height = 10  # pts
    # simple font
    c.setFont("Helvetica-Bold", 14)
    c.drawString(x, y, "Trader Behavior Insights — Report")
    y -= 1.5*line_height
    c.setFont("Helvetica", 10)
    y -= line_height
    wrap = textwrap.TextWrapper(width=100)
    # iterate markdown lines and write text; embed images when image filenames appear in outputs
    for i, raw in enumerate(lines):
        if raw.strip() == "":
            y -= line_height/2
            continue
        # If heading
        if raw.startswith("#"):
            level = raw.count("#")
            heading = raw.lstrip("#").strip()
            c.setFont("Helvetica-Bold", 12 - min(level-1,3))
            if y < 60:
                c.showPage()
                y = height - margin
            c.drawString(x, y, heading)
            y -= 1.2*line_height
            c.setFont("Helvetica", 10)
            continue
        # If image reference present (we don't require markdown image syntax; we will embed known output files)
        # embed each image from outputs folder in sequence at logical breakpoints (first few image files)
        # We'll attempt to place outputs in the order: profitability_by_sentiment.png, avg_profit_by_sentiment.png, returnpct_box_by_sentiment.png, buy_ratio_by_sentiment.png, sentiment_distribution.png, top15_accounts_avg_profit.png
        # Only embed once when encountering the word "charts" or "images" or "plots" or at end
        low = raw.lower()
        if any(k in low for k in ["charts", "images", "plots", "outputs", "figures"]):
            # embed images (loop)
            img_list = [
                "profitability_by_sentiment.png",
                "avg_profit_by_sentiment.png",
                "returnpct_box_by_sentiment.png",
                "buy_ratio_by_sentiment.png",
                "sentiment_distribution.png",
                "top15_accounts_avg_profit.png"
            ]
            c.setFont("Helvetica-Oblique", 9)
            y -= line_height/2
            for img in img_list:
                img_path = os.path.join(images_dir, img)
                if os.path.exists(img_path):
                    # scale image to page width
                    try:
                        im = ImageReader(img_path)
                        iw, ih = im.getSize()
                        scale = min((max_width) / iw, (height/3) / ih)
                        w = iw * scale
                        h = ih * scale
                        if y - h < margin:
                            c.showPage()
                            y = height - margin
                        c.drawImage(im, x, y-h, width=w, height=h)
                        y -= h + line_height/2
                    except Exception as e:
                        print("Could not embed", img_path, ":", e)
            c.setFont("Helvetica", 10)
            continue
        # otherwise normal paragraph (wrap long lines)
        wrapped = wrap.wrap(raw)
        for part in wrapped:
            if y < 60:
                c.showPage()
                y = height - margin
            c.drawString(x, y, part)
            y -= line_height
    c.save()
    print("Wrote PDF:", out_pdf)

# run
if not os.path.exists(MD_PATH):
    raise FileNotFoundError("ds_report.md not found at: " + MD_PATH)
write_pdf_from_md(MD_PATH, OUT_PDF, OUT_DIR)


Installing reportlab...



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Wrote PDF: c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\ds_report.pdf


In [27]:
import shutil, os
ROOT = os.path.dirname(os.getcwd())              # parent of notebooks
SRC_NOTEBOOK = os.path.join(ROOT, "notebooks", "Trader_Behavior_Analysis.ipynb")
DEST = os.path.join(ROOT, "ds_aditya_sawant", "notebook_1.ipynb")

if os.path.exists(SRC_NOTEBOOK):
    shutil.copyfile(SRC_NOTEBOOK, DEST)
    print("Copied notebook to:", DEST)
else:
    print("Source notebook not found at:", SRC_NOTEBOOK)
    print("If your notebook has a different name, set SRC_NOTEBOOK to that path and re-run.")


Copied notebook to: c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\notebook_1.ipynb


In [29]:
!dir ds_aditya_sawant


 Volume in drive C is Windows-SSD
 Volume Serial Number is 921E-3F6D

 Directory of c:\Users\Aditya\Desktop\datascience\notebooks



File Not Found


In [30]:
!dir "c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant"
!dir "c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\csv_files"
!dir "c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\outputs"


 Volume in drive C is Windows-SSD
 Volume Serial Number is 921E-3F6D

 Directory of c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant

24-11-2025  07:03 PM    <DIR>          .
24-11-2025  06:59 PM    <DIR>          ..
24-11-2025  06:59 PM    <DIR>          csv_files
24-11-2025  06:59 PM             1,566 ds_report.md
24-11-2025  07:02 PM           332,285 ds_report.pdf
24-11-2025  07:03 PM                 0 notebook_1.ipynb
24-11-2025  07:00 PM    <DIR>          outputs
24-11-2025  06:59 PM               382 README.md
               4 File(s)        334,233 bytes
               4 Dir(s)  14,680,162,304 bytes free
 Volume in drive C is Windows-SSD
 Volume Serial Number is 921E-3F6D

 Directory of c:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\csv_files

24-11-2025  06:59 PM    <DIR>          .
24-11-2025  07:03 PM    <DIR>          ..
24-11-2025  06:59 PM        62,257,978 final_ml_dataset.csv
24-11-2025  06:59 PM        60,929,492 merged_trades.csv
24-11-2025  06:59 PM        

In [1]:
import os

notebook_path = r"C:\Users\Aditya\Desktop\datascience\ds_aditya_sawant\notebook_1.ipynb"
print("Exists:", os.path.exists(notebook_path))
print("Size (bytes):", os.path.getsize(notebook_path))


Exists: True
Size (bytes): 0
